In [ ]:
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 10.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 90.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whe

In [16]:
# Импорт библиотек
import numpy as np
from datasets import Dataset
import nltk
from transformers import T5TokenizerFast, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
import torch
from transformers.optimization import Adafactor, AdafactorSchedule
from datasets import load_dataset, load_metric
import pandas as pd
from tqdm import tqdm

In [ ]:
# загрузка параметров
# raw_datasets = load_dataset("xsum")
metric = load_metric("rouge")
nltk.download('punkt')

<ipython-input-3-7eb3dae28fdb>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Ввести свой ключ huggingface hyb
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Определение параметров
REPO = "speller-t5-909"  # Введите наазвание название репозитория
MODEL_NAME =  "sberbank-ai/ruT5-large" # Введите наазвание выбранной модели из хаба
MAX_INPUT = 256  # Введите максимальную длинну входных данных  в токенах (длинна входных фраз в словах (можно считать полслова токен))
MAX_OUTPUT  = 256 # Введите максимальную длинну прогнозов в токенах (можно уменьшить для задач суммризации или других задач где выход короче)
BATCH_SIZE = 8

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
import pandas as pd
# data = pd.read_csv("./dataset_big.csv", index_col=0)[:15000]
data = pd.read_csv("./dataset_small.csv", index_col=0).drop_duplicates()
test_data = pd.read_csv("./test_dataset.csv", index_col=0).drop_duplicates()
data.head(2)

,source,target
0,очень классная тетка ктобы что не говорил.,очень классная тетка кто бы что ни говорил
1,Может выгоднее втулку продать и купить колесо ...,Может выгоднее втулку продать и купить колесо ...


In [ ]:
print(data.shape)
print(test_data.shape)

(117417, 2)
(2005, 2)


In [ ]:
import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from transformers import AutoTokenizer, AutoModel
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)
# model.resize_token_embeddings(len(tokenizer));
# model = nn.DataParallel(model).to(device);

In [ ]:
model.config.max_length = MAX_OUTPUT  # по умолчанию 20, поэтому во всех моделях прогнозы обрезаются выходные последовательности
# Закоментить после первого соъранения в репозиторий свой необъязательно
tokenizer.push_to_hub(REPO)

CommitInfo(commit_url='https://huggingface.co/summervent/speller-t5-909/commit/084f2a923846300cc2b92326e86769292bab88b5', commit_message='Upload tokenizer', commit_description='', oid='084f2a923846300cc2b92326e86769292bab88b5', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
data = pd.read_csv("./dataset_small.csv", index_col=0).drop_duplicates()

In [ ]:
input_data = list(data.source)
output_data = list(data.target)
task_prefix = "Исправь: "
input_sequences = input_data
encoding = tokenizer( [task_prefix + sequence for sequence in input_sequences], padding="longest", \
                     max_length=MAX_INPUT, truncation=True, return_tensors="pt", )
input_ids, attention_mask = encoding.input_ids, encoding.attention_mask
target_encoding = tokenizer(output_data, padding='max_length', max_length=MAX_OUTPUT, truncation=True)
labels = target_encoding.input_ids
labels = torch.tensor(labels)
labels[labels == tokenizer.pad_token_id] = -100
data = Dataset.from_pandas(pd.DataFrame({'input_ids': list(np.array(input_ids)), 'attention_mask': list(np.array(attention_mask)), \
                                         'labels': list(np.array(labels))}))
data = data.train_test_split(0.001)

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 117299
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 118
    })
})

In [ ]:
train = data['train']
test = data['test']

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) #return_tensors="tf"

In [ ]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  # Replace -100 in the labels as we can't decode them.
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  
  # Rouge expects a newline after each sentence
  decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
  decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
  
  result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  # Extract a few results
  result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
  
  # Add mean generated length
  prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  result["gen_len"] = np.mean(prediction_lens)
  
  return {k: round(v, 4) for k, v in result.items()}

In [ ]:
training_args = Seq2SeqTrainingArguments(
  output_dir = REPO,
  #overwrite_output_dir=True,
  evaluation_strategy='steps',
  #learning_rate=2e-5,
  logging_steps=10,
  eval_steps=1500,
  save_steps=5000,
  num_train_epochs=1,
  predict_with_generate=True,
  per_device_train_batch_size=BATCH_SIZE,
  per_device_eval_batch_size=BATCH_SIZE,
  fp16=True,
  save_total_limit=2,
  #generation_max_length=256,
  #generation_num_beams=4,
  weight_decay=0.005,
  # logging_strategy='steps',
  #logging_dir='logs',
  push_to_hub=True,
  report_to='tensorboard'
)

In [ ]:
optimizer = Adafactor(
    model.parameters(),
    lr=5e-5,
    eps=(1e-30, 1e-3),
    clip_threshold=1.0,
    decay_rate=-0.8,
    beta1=None,
    weight_decay=0.0,
    relative_step=False,
    scale_parameter=False,
    warmup_init=False,
)
lr_scheduler = AdafactorSchedule(optimizer)

In [ ]:
trainer = Seq2SeqTrainer(
  model=model,
  args=training_args,
  train_dataset = data['train'],
  eval_dataset = data['test'],
  optimizers = (optimizer, lr_scheduler),
  tokenizer = tokenizer,
  compute_metrics=compute_metrics,
)

In [ ]:
import logging
logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere
# or 
logging.disable(logging.WARNING)
trainer.train()   #продолжаем трейн на data

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1500,0.302200,0.156261,18.220300,5.932200,17.796600,18.220300,43.449200


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1500,0.302200,0.156261,18.220300,5.932200,17.796600,18.220300,43.449200
3000,0.227400,0.131133,18.220300,5.932200,17.796600,18.220300,42.381400
4500,0.200100,0.112778,18.220300,5.932200,17.796600,18.220300,41.940700
6000,0.175700,0.106299,18.220300,5.932200,17.796600,18.220300,42.254200
7500,0.161200,0.100201,17.937900,5.084700,17.514100,17.796600,42.339000
9000,0.171800,0.092138,18.220300,5.932200,17.796600,18.220300,42.050800
10500,0.167800,0.083440,17.796600,5.084700,17.372900,17.796600,41.983100
12000,0.140700,0.079335,18.220300,5.932200,17.796600,18.220300,42.211900
13500,0.144700,0.081358,18.220300,5.932200,17.796600,18.220300,42.042400


TrainOutput(global_step=14663, training_loss=0.19463932067653797, metrics={'train_runtime': 7643.3503, 'train_samples_per_second': 15.347, 'train_steps_per_second': 1.918, 'total_flos': 1.26978941386752e+17, 'train_loss': 0.19463932067653797, 'epoch': 1.0})

In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 32.0k/2.75G [00:00<?, ?B/s]

Upload file runs/Feb07_23-06-54_42bf987842d3/events.out.tfevents.1675812138.42bf987842d3.71504.2:  14%|#3     …

'https://huggingface.co/summervent/speller-t5-909/commit/e0bf920a049ba5c13f903996755e5ce7abd03289'

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

MODEL_NAME_ = "summervent/speller-t5-909" 
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME_).to(device);

In [ ]:
import pandas as pd
test_data = pd.read_csv("./test_dataset.csv", index_col=0).drop_duplicates()

In [27]:
MAX_INPUT = 256
input_sequences = list(test_data.source.values)

task_prefix = "Исправь: "
encoded = tokenizer(
    [task_prefix + sequence for sequence in input_sequences],
    padding="longest",
    max_length=MAX_INPUT,
    truncation=True,
    return_tensors="pt",
)
test_loader = torch.utils.data.DataLoader(input_sequences, batch_size=1)

predicts__ = []
i=0
loss = 0
for input_sequences in tqdm(test_loader):
    encoded = tokenizer(
        [task_prefix + sequence for sequence in input_sequences],
        padding="longest",
        max_length=MAX_INPUT,
        truncation=True,
        return_tensors="pt",
    )
    predicts_ = model.generate(**encoded.to(device),  max_length=len(encoded['input_ids'][0]))


    predicts_ = tokenizer.batch_decode(predicts_, skip_special_tokens=True)
    predicts__ += predicts_

100%|██████████| 2005/2005 [21:59<00:00,  1.52it/s]


In [29]:
idx = 913
predicts__[idx]

'из трех режимов наиболее полезными для меня оказались первый и последний то есть точки доступа и приемника в суровые времена локальных конфликтов и перекрытия линии связи вражескими силами можно будет подключаться к халявной точке доступа и обеспечивать себя связью с внешним миром'

In [30]:
test_data.reset_index().loc[913]['target']

'из трех режимов наиболее полезными для меня оказались первый и последний то есть точки доступа и приемника в суровые времена локальных конфликтов и перекрытия линии связи вражескими силами можно будет подключаться к халявной точке доступа и обеспечивать себя связью с внешним миром'

In [31]:
from evaluate_spell import extract_words, make_corrections_data, measure_quality


OUTPUT_DIFF = False


source_sents = [
    extract_words(line.strip().strip("\ufeff"))
    for line in test_data.source
    if line.strip().strip("\ufeff") != ""
]
correct_sents = [
    extract_words(line.strip().strip("\ufeff"))
    for line in test_data.target
    if line.strip().strip("\ufeff") != ""
]
answer_sents = [extract_words(line.strip().strip("\ufeff")) for line in predicts__]
etalon_corrections, answer_corrections = make_corrections_data(
    source_sents, correct_sents, answer_sents
)
TP, precision, recall, f_measure = measure_quality(
    etalon_corrections, answer_corrections
)
print(
    f"Precision={100*precision:.2f} Recall={100 * recall:.2f} FMeasure={100 * f_measure:.2f}"
)
print(TP, len(answer_corrections), len(etalon_corrections))
if OUTPUT_DIFF:
    output_differences(
        "diff.txt",
        source_sents,
        correct_sents,
        answer_sents,
        etalon_corrections,
        answer_corrections,
    );

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [32]:
count_correct = 0
for answer, correct in zip(correct_sents, answer_sents):
    if answer == correct:
        count_correct += 1
print(count_correct /len(test_data))

0.6074812967581047
